# Exercise 9

## Mashable news stories analysis

Predicting if a news story is going to be popular


In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/albahnsen/PracticalMachineLearningClass/master/datasets/mashable.csv'
df = pd.read_csv(url, index_col=0)
df.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,Popular
0,http://mashable.com/2014/12/10/cia-torture-rep...,28.0,9.0,188.0,0.732620,1.0,0.844262,5.0,1.0,1.0,...,0.200000,0.80,-0.487500,-0.60,-0.250000,0.9,0.8,0.4,0.8,1
1,http://mashable.com/2013/10/18/bitlock-kicksta...,447.0,7.0,297.0,0.653199,1.0,0.815789,9.0,4.0,1.0,...,0.160000,0.50,-0.135340,-0.40,-0.050000,0.1,-0.1,0.4,0.1,0
2,http://mashable.com/2013/07/24/google-glass-po...,533.0,11.0,181.0,0.660377,1.0,0.775701,4.0,3.0,1.0,...,0.136364,1.00,0.000000,0.00,0.000000,0.3,1.0,0.2,1.0,0
3,http://mashable.com/2013/11/21/these-are-the-m...,413.0,12.0,781.0,0.497409,1.0,0.677350,10.0,3.0,1.0,...,0.100000,1.00,-0.195701,-0.40,-0.071429,0.0,0.0,0.5,0.0,0
4,http://mashable.com/2014/02/11/parking-ticket-...,331.0,8.0,177.0,0.685714,1.0,0.830357,3.0,2.0,1.0,...,0.100000,0.55,-0.175000,-0.25,-0.100000,0.0,0.0,0.5,0.0,0


In [2]:
df.shape

(6000, 61)

In [3]:
df = df.drop('url', axis=1)

In [4]:
X = df.drop('Popular', axis=1)
y = df['Popular']

In [5]:
y.mean()

0.5

In [6]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Exercise 9.1

Estimate a Decision Tree Classifier and a Logistic Regresion

Evaluate using the following metrics:
* Accuracy
* F1-Score

### Decision Tree Classifier

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [8]:
#Árbol de clasificación con max_depth=10
treeclass = DecisionTreeClassifier(max_depth=10, random_state=1)
treeclass.fit(X_train, y_train)
y_pred = treeclass.predict(X_test)

In [9]:
# Evaluar F1-score y accuracy
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.5879299156391953, 0.5766666666666667)

### Logistic Regression

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [12]:
# Evaluar F1-score y accuracy
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6104529616724739, 0.6273333333333333)

# Exercise 9.2

Estimate 300 bagged samples

Estimate the following set of classifiers:

* 100 Decision Trees where max_depth=None
* 100 Decision Trees where max_depth=2
* 100 Logistic Regressions

In [13]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingRegressor

C:\Users\aparrado\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [14]:
train = X_train.join(y_train, rsuffix='y_train')
test =  X_test.join(y_test, rsuffix='y_test')

In [15]:
# muestra y cantidad de muestras
np.random.seed(123) # Semilla
n_samples = train.shape[0]
n_B = 300
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

### Decision Tree max depth=None

In [16]:
bagreg_n = BaggingRegressor(DecisionTreeRegressor(max_depth=None, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

bagreg_n.fit(X_train, y_train)
y_pred = bagreg_n.predict(X_test)

In [17]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.46895493742291844

### Decision Tree max depth=2

In [18]:
bagreg_2 = BaggingRegressor(DecisionTreeRegressor(max_depth=2, random_state=123), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)

bagreg_2.fit(X_train, y_train)
y_pred = bagreg_2.predict(X_test)

In [19]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.47104640971845413

### Logistic Regression

In [20]:
bagreg_log = BaggingRegressor(LogisticRegression(), n_estimators=100, 
                          bootstrap=True, oob_score=True, random_state=1)
bagreg_log.fit(X_train, y_train)
y_pred = bagreg_log.predict(X_test)

In [21]:
np.sqrt(mean_squared_error(y_test, y_pred))

0.5812533584132368

# Exercise 9.3

Ensemble using majority voting

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [22]:
from sklearn.ensemble import BaggingClassifier

In [23]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [24]:
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6456278763971072, 0.6406666666666667)

# Exercise 9.4

Estimate te probability as % models that predict positive

Modify the probability threshold and select the one that maximizes the F1-Score

In [25]:
# Probabilidad
clf.predict_proba(X_test)

array([[0.22, 0.78],
       [0.39, 0.61],
       [0.42, 0.58],
       ...,
       [0.37, 0.63],
       [0.65, 0.35],
       [0.72, 0.28]])

In [ ]:
probs = ([0.01, 0.02])
F1_scores = []

for i in probs:
    clf = DecisionTreeClassifier(random_state = 2)
    clf.fit(X_train, y_train)
    y_pred[i] = (clf.predict_proba(X_test)
    F1_scores = metrics.f1_score(y_pred[i], y_test)

# Exercise 9.5

Ensemble using weighted voting using the oob_error

Evaluate using the following metrics:
* Accuracy
* F1-Score

In [26]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [27]:
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6456278763971072, 0.6406666666666667)

In [28]:
errors = np.zeros(clf.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], clf.n_estimators))

for i in range(clf.n_estimators):
    oob_sample = ~clf.estimators_samples_[i]
    y_pred_ = clf.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = clf.estimators_[i].predict(X_test)
    
alpha = (1 - errors) / (1 - errors).sum()
y_pred = (np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int)

In [29]:
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.6497064579256361, 0.642)

# Exercise 9.6

Estimate te probability of the weighted voting

Modify the probability threshold and select the one that maximizes the F1-Score

In [30]:
# Probabilidad
clf.predict_proba(X)

array([[0.16, 0.84],
       [0.95, 0.05],
       [0.9 , 0.1 ],
       ...,
       [0.95, 0.05],
       [0.83, 0.17],
       [0.11, 0.89]])

In [ ]:
probs = ([0.01, 0.02])
F1_scores = []

for i in probs:
    clf = DecisionTreeClassifier(random_state = 2)
    clf.fit(X_train, y_train)
    y_pred[i] = (clf.predict_proba(X_test)
    F1_scores = metrics.f1_score(y_pred[i], y_test)

# Exercise 9.7

Estimate a logistic regression using as input the estimated classifiers

Modify the probability threshold such that maximizes the F1-Score